### 整合BiGG、KEGG、BioCyc、MetaNetX、Rhea、MetaCyc、ModelSEED这些数据库的反应和代谢物信息，从而整合BiGG、KEGG、MetaCyc的反应作为反应库，构建一个反应模型，命名为BKMD(bigg_kegg_metacyc_database)

#### BiGG、Rhea、MetaNetX、MetaCyc、ModelSEED这几个数据库均提供了反应和代谢物的对应信息，可以分别提取出来，再进行整合

In [1]:
# 数据库原始数据获取方式
# 具体内容可参见Gitlab: http://39.105.210.119/model-construction-analysis/pathway-design/tree/Yu_wt/Database_Map
# BiGG: 网站右上角 Data Access,下载文件 bigg_models_reactions.txt 和 bigg_models_metabolites.txt，从中提取反应代谢物对应信息；
#下载 universal_model.json 作为 BiGG 的部分的反应集
# KEGG: 利用 bioservices package 下载KEGG的反应和代谢物信息
# Rhea: 网站 Download ,在页面最后的 Cross-references 中下载 rhea2xrefs.tsv文件，从中筛选对应关系
# MetaCyc: http://bioinformatics.ai.sri.com/ptools/dist-473256968.html 网站下载 Pathway Tools Software ; 打开软件，选取MetaCyc,
# File-Export-Generate SBML File For-Selected Reactions-选取存储路径-文件命名-OK
# ModelSEED: https://github.com/ModelSEED/ModelSEEDDatabase , Biochemistry目录下下载 compounds.tsv 和 reactions.tsv 文件，从中提取反应代谢物对应关系
# MetaNetX: 网站左边点击 Search/Download MNXref namespace , 点击 Previous releases 下的 MNXref FTP ,进入最先版本的文件夹（此处4.0），下载相关文件

In [2]:
import pandas as pd
import csv 
import re
from functools import partial, reduce

In [3]:
# 数据库文件存储位置
inputforder = '/home/liaoxp/Notes/yu_wt/PathwayFind/database_mapping/Database_info/'

### BiGG

In [4]:
# BiGG数据库反应对应关系提取
df_bg_rxn = pd.read_table(inputforder+'bigg_models_reactions.txt')  # 存储反应信息的文件
df_bg_met = pd.read_table(inputforder+'bigg_models_metabolites.txt')  # 存储代谢物信息的文件
#df_bg_rxn
bg_rxn_map = []  # 存储反应对应关系
bg_rxn_nrows = df_bg_rxn.shape[0]  # 反应文件的总行数
bg_met_map = []  # 存储代谢物对应信息
bg_met_nrows = df_bg_met.shape[0]  # 代谢物文件的总行数
for i in range(bg_rxn_nrows):
    rhea_rxn = re.findall(r"rhea/([\s\S]+?);",str(df_bg_rxn.iat[i,4]))   # 反应对应的Rhea信息提取
    kegg_rxn = re.findall(r"kegg.reaction/(R\d{5})",str(df_bg_rxn.iat[i,4]))  # 反应对应的KEGG信息提取
    biocyc_rxn = re.findall(r"META:([\s\S]+?);",str(df_bg_rxn.iat[i,4]))  # 反应对应的BioCyc信息提取
    mnx_rxn = re.findall(r"MNXR\d*",str(df_bg_rxn.iat[i,4]))  # 反应对应的MetaNetX信息提取
    seed_rxn = re.findall(r"rxn\d*",str(df_bg_rxn.iat[i,4]))  # 反应对应的ModelSEED信息提取
    temp = []
    temp.append(df_bg_rxn.iat[i,0])  # BiGG反应id
    temp.append(df_bg_rxn.iat[i,1])  # BiGG反应name
    temp.append(df_bg_rxn.iat[i,2])  # BiGG反应equation
    temp.append(", ".join(rhea_rxn))
    temp.append(", ".join(kegg_rxn))
    temp.append(", ".join(biocyc_rxn))
    temp.append(", ".join(mnx_rxn))
    temp.append(", ".join(seed_rxn))
    bg_rxn_map.append(temp)

# BiGG数据库代谢物对应关系提取
#df_bg_met
for i in range(bg_met_nrows):
    kegg_met = re.findall(r"kegg.*/([A-Z]\d{5}|[a-z]\d{5})",str(df_bg_met.iat[i,4]))  # 代谢物对应的KEGG信息提取
    biocyc_met = re.findall(r"META:([\s\S]+?);",str(df_bg_met.iat[i,4]))  # 代谢物对应的BioCyc信息提取
    mnx_met = re.findall(r"MNXM\d*",str(df_bg_met.iat[i,4]))  # 代谢物对应的MetaNetX信息提取
    seed_met = re.findall(r"cpd\d*",str(df_bg_met.iat[i,4]))  # 代谢物对应的ModelSEED信息提取
    temp = []
    temp.append(df_bg_met.iat[i,0])  # BiGG代谢物 id
    temp.append(df_bg_met.iat[i,1])  # BiGG代谢物 universal_id（去除分室信息）
    temp.append(df_bg_met.iat[i,2])  # BiGG代谢物 name
    temp.append(", ".join(kegg_met))
    temp.append(", ".join(biocyc_met))
    temp.append(", ".join(mnx_met))
    temp.append(", ".join(seed_met))
    bg_met_map.append(temp)
    
writer = pd.ExcelWriter('BiGGdata_map.xlsx')
pd.DataFrame(bg_met_map).to_excel(writer, 'metmapping',header=['BiGG ID','universal_id','Name','Kegg','Biocyc','MNX','Seed'],index=False)
pd.DataFrame(bg_rxn_map).to_excel(writer, 'rxnmapping',header=['BiGG ID','Name','Equation','Rhea','Kegg','Biocyc','MNX','Seed'],index=False)
writer.save()

### MetaCyc

In [5]:
# MetaCyc数据库代谢物对应关系提取 输入文件名只出现一次
df_rxns = pd.read_excel(inputforder+'MetaCyc_24_pt.xlsx',sheet_name = 'reactions')  # MetaCyc反应信息
df_mets = pd.read_excel(inputforder+'MetaCyc_24_pt.xlsx',sheet_name = 'metabolites')  # MetaCyc代谢物信息
nrows_m = df_mets.shape[0]  # 代谢物dataframe总行数
mets_map = []  # 存储代谢物对应关系
nrows_r = df_rxns.shape[0]  # 反应dataframe总行数
rxns_map = []  # 存储反应对应关系
for i in range(nrows_m):
    temp = []
    biocyc_m = re.findall(r"META:([\s\S]+?),",str(df_mets.iat[i,6]))  # 代谢物对应的BioCyc信息提取
    kegg_m = re.findall(r"kegg.*/([A-Z]\d{5}|[a-z]\d{5})",str(df_mets.iat[i,6]))  # 代谢物对应的KEGG信息提取
    temp.append(df_mets.iat[i,0])
    temp.append(", ".join(biocyc_m))
    temp.append(", ".join(kegg_m))
    mets_map.append(temp)

# MetaCyc数据库反应对应关系提取
for j in range(nrows_r):
    Temp = []
    biocyc_r = re.split(r'[\[\(\)\]\',\'\s]+',str(re.findall(r"META:([\s\S]+?),|META:([\s\S]+?)$",str(df_rxns.iat[j,5]))))  # 反应对应的BioCyc信息提取
    for x in range(biocyc_r.count('')):
        biocyc_r.remove('')
    kegg_r = re.findall(r"kegg.reaction/([A-Z]\d{5})",str(df_rxns.iat[j,5]))  # 反应对应的KEGG信息提取
    rhea_r = re.findall(r"rhea/(\d{5})",str(df_rxns.iat[j,5]))  # 反应对应的Rhea信息提取
    Temp.append(df_rxns.iat[j,0])
    Temp.append(", ".join(biocyc_r))
    Temp.append(", ".join(kegg_r))
    Temp.append(", ".join(rhea_r))
    rxns_map.append(Temp)
    
writer = pd.ExcelWriter('MetaCycdata_map.xlsx')
pd.DataFrame(mets_map).to_excel(writer, 'metmapping',header=['MetaCyc','BioCyc','KEGG'],index=False)
pd.DataFrame(rxns_map).to_excel(writer, 'rxnmapping',header=['MetaCyc','BioCyc','KEGG','Rhea'],index=False)
writer.save()

### ModelSEED

In [6]:
# ModelSEED数据库反应对应关系提取
df_seed_rxn = pd.read_table(inputforder+'seed_reactions.tsv')  # ModelSEED反应信息
df_seed_met = pd.read_table(inputforder+'seed_compounds.tsv')  # ModelSEED代谢物信息
#df_seed_rxn
seed_rxn_map = []  # 存储反应对应信息
seed_rxn_nrows = df_seed_rxn.shape[0]  # 反应dataframe总行数
seed_met_map = []  # 存储代谢物对应信息
seed_met_nrows = df_seed_met.shape[0]  # 代谢物dataframe总行数

for i in range(seed_rxn_nrows):
    bigg_rxn = re.findall(r"BiGG:\s([\s\S]+?)\|",str(df_seed_rxn.iat[i,12]))  # 反应对应的BiGG信息提取
    kegg_rxn = re.split(r'[\[\(\)\]\';,\'\s]+',str(re.findall(r"KEGG:\s([\s\S]+?)\||KEGG:\s([\s\S]+?)$",str(df_seed_rxn.iat[i,12]))))  # 反应对应的KEGG信息提取
    for x in range(kegg_rxn.count('')):
        kegg_rxn.remove('')
    metacyc_rxn = re.split(r'[\[\(\)\]\';,\'\s]+',str(re.findall(r"MetaCyc:\s([\s\S]+?)\||MetaCyc:\s([\s\S]+?)$",str(df_seed_rxn.iat[i,12]))))  # 反应对应的MetaCyc信息提取
    for y in range(metacyc_rxn.count('')):
        metacyc_rxn.remove('')
    temp = []
    temp.append(df_seed_rxn.iat[i,0])  # SEED反应id
    temp.append(df_seed_rxn.iat[i,2])  # SEED反应name
    temp.append(", ".join(bigg_rxn))
    temp.append(", ".join(kegg_rxn))
    temp.append(", ".join(metacyc_rxn))
    seed_rxn_map.append(temp)

# ModelSEED数据库代谢物对应信息提取
#df_seed_met
for i in range(seed_met_nrows):
    bigg_met = re.split(r'[\[\(\)\]\';,\'\s]+',str(re.findall(r"BiGG:\s([\s\S]+?)\||BiGG:\s([\s\S]+?)$",str(df_seed_met.iat[i,18]))))  # 代谢物对应的BiGG信息
    for x in range(bigg_met.count('')):
        bigg_met.remove('')
    kegg_met = re.split(r'[\[\(\)\]\';,\'\s]+',str(re.findall(r"KEGG:\s([\s\S]+?)\||KEGG:\s([\s\S]+?)$",str(df_seed_met.iat[i,18]))))  # 代谢物对应的KEGG信息
    for y in range(kegg_met.count('')):
        kegg_met.remove('')
    metacyc_met = re.split(r'[\[\(\)\]\';,\'\s]+',str(re.findall(r"MetaCyc:\s([\s\S]+?)\||MetaCyc:\s([\s\S]+?)$",str(df_seed_met.iat[i,18]))))  # 代谢物对应的MetaCyc信息
    for z in range(metacyc_met.count('')):
        metacyc_met.remove('')
    temp = []
    temp.append(df_seed_met.iat[i,0])  # SEED代谢物 id
    temp.append(df_seed_met.iat[i,2])  # SEED代谢物 name
    temp.append(", ".join(bigg_met))
    temp.append(", ".join(kegg_met))
    temp.append(", ".join(metacyc_met))
    seed_met_map.append(temp)
    
writer = pd.ExcelWriter('SEEDdata_map.xlsx')
pd.DataFrame(seed_met_map).to_excel(writer, 'metmapping',header=['SEED ID','Name','Bigg','Kegg','Metacyc'],index=False)
pd.DataFrame(seed_rxn_map).to_excel(writer, 'rxnmapping',header=['SEED ID','Name','Bigg','Kegg','Metacyc'],index=False)
writer.save()

/home/liaoxp/anaconda3/envs/enzyme_python/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (4,10,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### MetaNetX

In [7]:
# 处理 MetaNetX 下载的原始数据，删除其中的注释行，直接覆盖原文件

with open(inputforder+'mnx_reac_xref.tsv', 'r+') as f:  # 存储MetaNetX反应对应信息的文件
    p = re.compile('#')  # 注释行以‘#’开头
    lines = [line for line in f.readlines() if p.match(line) is None]
    f.seek(0)
    f.truncate(0)
    f.writelines(lines)
    
with open(inputforder+'mnx_chem_xref.tsv', 'r+') as f:  # 存储MetaNetX代谢物对应信息的文件
    p = re.compile('#')
    lines = [line for line in f.readlines() if p.match(line) is None]
    f.seek(0)
    f.truncate(0)
    f.writelines(lines)

with open(inputforder+'mnx_reac_depr.tsv', 'r+') as f:  # 存储MetaNetX反应ID版本迭代信息的文件
    p = re.compile('#')
    lines = [line for line in f.readlines() if p.match(line) is None]
    f.seek(0)
    f.truncate(0)
    f.writelines(lines)
    
with open(inputforder+'mnx_chem_depr.tsv', 'r+') as f:  # 存储MetaNetX代谢物ID版本迭代信息的文件
    p = re.compile('#')
    lines = [line for line in f.readlines() if p.match(line) is None]
    f.seek(0)
    f.truncate(0)
    f.writelines(lines)

In [16]:
# 删除反应和代谢物信息中没有MetaNetX ID或没有和其他数据库对应信息的行
df_mnxrxn_origin = pd.read_table(inputforder+'mnx_reac_xref.tsv', header=None)  # 删除注释信息后的反应对应关系文件
df_mnxmet_origin = pd.read_table(inputforder+'mnx_chem_xref.tsv', header=None)  # 删除注释信息后的代谢物对应关系文件
df_mnxrxn_origin.columns = ['Source','MNX ID','Description']  # 加行索引
df_mnxmet_origin.columns = ['Source','MNX ID','Description']  # 加行索引
nrows_rxn = df_mnxrxn_origin.shape[0]  # 反应dataframe总行数
del_row_rxn = []  # 存储反应dataframe需要删除行的索引
nrows_met = df_mnxmet_origin.shape[0]  # 代谢物dataframe总行数
del_row_met = []  # 存储代谢物dataframe需要删除行的索引
mnx_bigg_rxn = {}  # 存储MetaNetX和BiGG反应的对应关系
mnx_kegg_rxn = {}  # 存储MetaNetX和KEGG反应的对应关系
mnx_metacyc_rxn = {}  # 存储MetaNetX和MetaCyc反应的对应关系
mnx_rhea_rxn = {}  # 存储MetaNetX和Rhea反应的对应关系
mnx_seed_rxn = {}  # 存储MetaNetX和SEED反应的对应关系
mnx_bigg_met = {}  # 存储MetaNetX和BiGG代谢物的对应关系
mnx_kegg_met = {}  # 存储MetaNetX和KEGG代谢物的对应关系
mnx_metacyc_met = {}  # 存储MetaNetX和MetaCyc代谢物的对应关系
mnx_rhea_met = {}  # 存储MetaNetX和Rhea代谢物的对应关系
mnx_seed_met = {}  # 存储MetaNetX和SEED代谢物的对应关系

for i in range(nrows_rxn):
    if df_mnxrxn_origin.iat[i,1] == 'EMPTY':  # 没有MetaNetX反应ID的行
        del_row_rxn.append(i)
    if not re.search(':',str(df_mnxrxn_origin.iat[i,0])):  # 没有MetaNetX反应对应关系的行
        del_row_rxn.append(i)
df_mnxrxn_origin.drop(del_row_rxn,inplace=True)  # 删除符合条件行后的新的dataframe

for j in range(nrows_met):
    if df_mnxmet_origin.iat[j,1] == 'EMPTY':  # 没有MetaNetX代谢物ID的行
        del_row_met.append(j)
    if not re.search(':',str(df_mnxmet_origin.iat[j,0])):  # 没有MetaNetX代谢物对应关系的行
        del_row_met.append(j)
df_mnxmet_origin.drop(del_row_met,inplace=True)  # 删除符合条件行后的新的dataframe

newdf_rxn = pd.DataFrame(df_mnxrxn_origin, columns=['MNX ID', 'Source'])  # 提取原始数据中的这两列
newdf_rxn['ref'] = newdf_rxn['Source'].map(lambda x:x.split(':')[0])  # 将Source列根据':'分割为两列，这一列为参考数据库
newdf_rxn['id'] = newdf_rxn['Source'].map(lambda x:x.split(':')[1])  # 将Source列根据':'分割为两列，这一列为相应ID

newdf_met = pd.DataFrame(df_mnxmet_origin, columns=['MNX ID', 'Source'])  # 提取原始数据中的这两列
newdf_met['ref'] = newdf_met['Source'].map(lambda x:x.split(':')[0])  # 将Source列根据':'分割为两列，这一列为参考数据库
newdf_met['id'] = newdf_met['Source'].map(lambda x:x.split(':')[1])  # 将Source列根据':'分割为两列，这一列为相应ID
# 对应关系提取
# 处理反应
n = newdf_rxn.shape[0]  # 新的反应dataframe总行数
for i in range(n):
    if newdf_rxn.iat[i,2] == 'biggR':  # 反应对应的BiGG信息提取
        if newdf_rxn.iat[i,0] in mnx_bigg_rxn:
            mnx_bigg_rxn.get(newdf_rxn.iat[i,0]).append(newdf_rxn.iat[i,3]) 
        else:
            mnx_bigg_rxn.setdefault(newdf_rxn.iat[i,0],[]).append(newdf_rxn.iat[i,3])
    if newdf_rxn.iat[i,2] == 'keggR':  # 反应对应的KEGG信息提取
        if newdf_rxn.iat[i,0] in mnx_kegg_rxn:
            mnx_kegg_rxn.get(newdf_rxn.iat[i,0]).append(newdf_rxn.iat[i,3]) 
        else:
            mnx_kegg_rxn.setdefault(newdf_rxn.iat[i,0],[]).append(newdf_rxn.iat[i,3])
    if newdf_rxn.iat[i,2] == 'metacycR':  # 反应对应的MetaCyc信息提取
        if newdf_rxn.iat[i,0] in mnx_metacyc_rxn:
            mnx_metacyc_rxn.get(newdf_rxn.iat[i,0]).append(newdf_rxn.iat[i,3]) 
        else:
            mnx_metacyc_rxn.setdefault(newdf_rxn.iat[i,0],[]).append(newdf_rxn.iat[i,3])
    if newdf_rxn.iat[i,2] == 'rheaR':  # 反应对应的Rhea信息提取
        if newdf_rxn.iat[i,0] in mnx_rhea_rxn:
            mnx_rhea_rxn.get(newdf_rxn.iat[i,0]).append(newdf_rxn.iat[i,3]) 
        else:
            mnx_rhea_rxn.setdefault(newdf_rxn.iat[i,0],[]).append(newdf_rxn.iat[i,3])
    if newdf_rxn.iat[i,2] == 'seedR':  # 反应对应的SEDD信息提取
        if newdf_rxn.iat[i,0] in mnx_seed_rxn:
            mnx_seed_rxn.get(newdf_rxn.iat[i,0]).append(newdf_rxn.iat[i,3]) 
        else:
            mnx_seed_rxn.setdefault(newdf_rxn.iat[i,0],[]).append(newdf_rxn.iat[i,3])
# 将这五个字典转成dataframe并合并
df1_r = pd.DataFrame(list(mnx_bigg_rxn.items()), columns=['MNX_id', 'BiGG_id'])
df2_r = pd.DataFrame(list(mnx_kegg_rxn.items()), columns=['MNX_id', 'KEGG_id'])
df3_r = pd.DataFrame(list(mnx_metacyc_rxn.items()), columns=['MNX_id', 'MetaCyc_id'])
df4_r = pd.DataFrame(list(mnx_rhea_rxn.items()), columns=['MNX_id', 'Rhea_id'])
df5_r = pd.DataFrame(list(mnx_seed_rxn.items()), columns=['MNX_id', 'SEED_id'])
df1_r.set_index(["MNX_id"], inplace=True)  # 设置MNX_id为索引，方便合并
df2_r.set_index(["MNX_id"], inplace=True)
df3_r.set_index(["MNX_id"], inplace=True)
df4_r.set_index(["MNX_id"], inplace=True)
df5_r.set_index(["MNX_id"], inplace=True)
dfs_r = [df1_r,df2_r,df3_r,df4_r,df5_r]
df_rxn_map = pd.concat(dfs_r,axis=1,join='outer',sort='False')
df_rxn_map = df_rxn_map.reset_index()  # 恢复原索引
df_rxn_map.columns = ['MNX_id','BiGG_id','KEGG_id','MetaCyc_id','Rhea_id','SEED_id']
# 去除单元格中的中括号，列表使用下列代码，字符串使用 df['BiGG_id'] = df['BiGG_id'].str.strip('[]')
df_rxn_map['BiGG_id'] = df_rxn_map['BiGG_id'].str.join(',')
df_rxn_map['KEGG_id'] = df_rxn_map['KEGG_id'].str.join(',')
df_rxn_map['MetaCyc_id'] = df_rxn_map['MetaCyc_id'].str.join(',')
df_rxn_map['Rhea_id'] = df_rxn_map['Rhea_id'].str.join(',')
df_rxn_map['SEED_id'] = df_rxn_map['SEED_id'].str.join(',')
# 处理代谢物
n1 = newdf_met.shape[0]  # 新的代谢物dataframe总行数
for j in range(n1):
    if re.search('bigg',str(newdf_met.iat[j,2])):  # 代谢物对应的BiGG信息提取
        if newdf_met.iat[j,0] in mnx_bigg_met:
            mnx_bigg_met.get(newdf_met.iat[j,0]).append(newdf_met.iat[j,3]) 
        else:
            mnx_bigg_met.setdefault(newdf_met.iat[j,0],[]).append(newdf_met.iat[j,3])
    if re.search('kegg',str(newdf_met.iat[j,2])):  # 代谢物对应的KEGG信息提取
        if newdf_met.iat[j,0] in mnx_kegg_met:
            mnx_kegg_met.get(newdf_met.iat[j,0]).append(newdf_met.iat[j,3]) 
        else:
            mnx_kegg_met.setdefault(newdf_met.iat[j,0],[]).append(newdf_met.iat[j,3])
    if re.search('metacyc',str(newdf_met.iat[j,2])):  # 代谢物对应的MetaCyc信息提取
        if newdf_met.iat[j,0] in mnx_metacyc_met:
            mnx_metacyc_met.get(newdf_met.iat[j,0]).append(newdf_met.iat[j,3]) 
        else:
            mnx_metacyc_met.setdefault(newdf_met.iat[j,0],[]).append(newdf_met.iat[j,3])
    if re.search('rhea',str(newdf_met.iat[j,2])):  # 代谢物对应的Rhea信息提取
        if newdf_met.iat[j,0] in mnx_rhea_met:
            mnx_rhea_met.get(newdf_met.iat[j,0]).append(newdf_met.iat[j,3]) 
        else:
            mnx_rhea_met.setdefault(newdf_met.iat[j,0],[]).append(newdf_met.iat[j,3])
    if re.search('seed',str(newdf_met.iat[j,2])):  # 代谢物对应的SEED信息提取
        if newdf_met.iat[j,0] in mnx_seed_met:
            mnx_seed_met.get(newdf_met.iat[j,0]).append(newdf_met.iat[j,3]) 
        else:
            mnx_seed_met.setdefault(newdf_met.iat[j,0],[]).append(newdf_met.iat[j,3])
# 将这五个字典转成dataframe并合并
df1_m = pd.DataFrame(list(mnx_bigg_met.items()), columns=['MNX_id', 'BiGG_id'])
df2_m = pd.DataFrame(list(mnx_kegg_met.items()), columns=['MNX_id', 'KEGG_id'])
df3_m = pd.DataFrame(list(mnx_metacyc_met.items()), columns=['MNX_id', 'MetaCyc_id'])
df4_m = pd.DataFrame(list(mnx_rhea_met.items()), columns=['MNX_id', 'Rhea_id'])
df5_m = pd.DataFrame(list(mnx_seed_met.items()), columns=['MNX_id', 'SEED_id'])
df1_m.set_index(["MNX_id"], inplace=True)  # 设置MNX_id为索引，方便合并
df2_m.set_index(["MNX_id"], inplace=True)
df3_m.set_index(["MNX_id"], inplace=True)
df4_m.set_index(["MNX_id"], inplace=True)
df5_m.set_index(["MNX_id"], inplace=True)
dfs_m = [df1_m,df2_m,df3_m,df4_m,df5_m]
df_met_map = pd.concat(dfs_m,axis=1,join='outer',sort='False')
df_met_map = df_met_map.reset_index()  # 恢复原索引
df_met_map.columns = ['MNX_id','BiGG_id','KEGG_id','MetaCyc_id','Rhea_id','SEED_id']
# 去除单元格中的中括号，列表使用下列代码，字符串使用 df['BiGG_id'] = df['BiGG_id'].str.strip('[]')
df_met_map['BiGG_id'] = df_met_map['BiGG_id'].str.join(',')
df_met_map['KEGG_id'] = df_met_map['KEGG_id'].str.join(',')
df_met_map['MetaCyc_id'] = df_met_map['MetaCyc_id'].str.join(',')
df_met_map['Rhea_id'] = df_met_map['Rhea_id'].str.join(',')
df_met_map['SEED_id'] = df_met_map['SEED_id'].str.join(',')

In [40]:
ID_deprID_rxn = {}  # 反应的现用ID和弃用ID信息
ID_deprID_met = {}  # 代谢物的现用ID和弃用ID信息
df_mnxmet_depr = pd.read_table(inputforder+'mnx_chem_depr.tsv',header=None)  # MetaNetX代谢物ID版本迭代信息
df_mnxmet_depr.columns = ['deprecated_ID','MNX_id','Version']  # 加行索引
df_mnxrxn_depr = pd.read_table(inputforder+'mnx_reac_depr.tsv',header=None)  # MetaNetX反应ID版本迭代信息
df_mnxrxn_depr.columns = ['deprecated_ID','MNX_id','Version']  # 加行索引
# 需要合并，一个新的ID有多个弃用的ID
n_r = df_mnxrxn_depr.shape[0]
n_m = df_mnxmet_depr.shape[0]
for i in range(n_r):
    if df_mnxrxn_depr.iat[i,1] in ID_deprID_rxn:
        ID_deprID_rxn.get(df_mnxrxn_depr.iat[i,1]).append(df_mnxrxn_depr.iat[i,0]) 
    else:
        ID_deprID_rxn.setdefault(df_mnxrxn_depr.iat[i,1],[]).append(df_mnxrxn_depr.iat[i,0])
for j in range(n_m):
    if df_mnxmet_depr.iat[j,1] in ID_deprID_met:
        ID_deprID_met.get(df_mnxmet_depr.iat[j,1]).append(df_mnxmet_depr.iat[j,0]) 
    else:
        ID_deprID_met.setdefault(df_mnxmet_depr.iat[j,1],[]).append(df_mnxmet_depr.iat[j,0]) 
        
df_r = pd.DataFrame(list(ID_deprID_rxn.items()), columns=['MNX_id', 'deprecated_ID'])
df_m = pd.DataFrame(list(ID_deprID_met.items()), columns=['MNX_id', 'deprecated_ID'])
df_r['deprecated_ID'] = df_r['deprecated_ID'].str.join(',')
df_m['deprecated_ID'] = df_m['deprecated_ID'].str.join(',')

newdf_rxn_map = pd.merge(df_rxn_map,df_r,how='left',on='MNX_id')   # 添加反应ID迭代信息
newdf_met_map = pd.merge(df_met_map,df_m,how='left',on='MNX_id')  # 添加代谢物ID迭代信息
order = ['MNX_id', 'deprecated_ID', 'BiGG_id', 'KEGG_id', 'MetaCyc_id', 'Rhea_id', 'SEED_id']  # 调整列顺序
newdf_rxn_map = newdf_rxn_map[order]
newdf_met_map = newdf_met_map[order]

writer = pd.ExcelWriter('MNXdata_map.xlsx')
newdf_met_map.to_excel(writer, 'metmapping',header=['MNX ID','deprecated_ID','Bigg','Kegg','Metacyc','Rhea','SEED'],index=False)
newdf_rxn_map.to_excel(writer, 'rxnmapping',header=['MNX ID','deprecated_ID','Bigg','Kegg','Metacyc','Rhea','SEED'],index=False)
writer.save()

### Rhea

In [18]:
# Rhea数据库
df_rhea_rxn = pd.read_table(inputforder+'rhea2xrefs.tsv')
newdf = df_rhea_rxn.query('DB == "KEGG_REACTION" | DB == "METACYC"')  # 仅保留有KEGG和MetaCyc对应关系的行
nrows = newdf.shape[0]
rhea_metacyc = {}  # Rhea 和 MetaCyc的对应关系
rhea_kegg = {}  # Rhea 和 KEGG的对应关系
for i in range(nrows):
    if newdf.iat[i,4] == 'METACYC':
        if newdf.iat[i,2] in rhea_metacyc:
            rhea_metacyc.get(newdf.iat[i,2]).append(newdf.iat[i,3]) 
        else:
            rhea_metacyc.setdefault(newdf.iat[i,2],[]).append(newdf.iat[i,3])
    if newdf.iat[i,4] == 'KEGG_REACTION':
        if newdf.iat[i,2] in rhea_kegg:
            rhea_kegg.get(newdf.iat[i,2]).append(newdf.iat[i,3]) 
        else:
            rhea_kegg.setdefault(newdf.iat[i,2],[]).append(newdf.iat[i,3])
# 字典转dataframe，并合并            
df1 = pd.DataFrame(list(rhea_metacyc.items()), columns=['Rhea', 'MetaCyc'])
df2 = pd.DataFrame(list(rhea_kegg.items()), columns=['Rhea', 'KEGG'])
df = pd.merge(df1,df2,how='outer',on='Rhea')
# 去除单元格的中括号
df['MetaCyc'] = df['MetaCyc'].str.join(',')
df['KEGG'] = df['KEGG'].str.join(',')

writer = pd.ExcelWriter('Rheadata_map.xlsx')
df.to_excel(writer, 'rxnmapping',header=['Rhea','Metacyc','Kegg'],index=False)
writer.save()

### 根据人工修正的对应关系修改相关信息

### 各数据库数据整合